In [89]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import (Input, GlobalAveragePooling2D, Dense, Concatenate, 
                                     TimeDistributed, GlobalAveragePooling1D, Dropout)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.layers import Lambda

In [59]:
#############################################
# 1. ПАРСИНГ МЕТОК condition
#############################################

# Списки/словарь допустимых значений
# Для монет с дефектами (строки, содержащие "det")
defective_mapping = {
    "PF det.": 0,
    "UNC det.": 1,
    "AU det.": 2,
    "XF det.": 3,
    "VF det.": 4,
    "F det.": 5,
    "VG det.": 6,
    "G det.": 7
}

# Для монет без дефектов: допустимые числовые оценки
no_def_allowed = [4,6,8,10,12,15,20,25,30,35,40,45,50,53,55,58,60,61,62,63,64,65,66,67,68,69,70]
no_def_mapping = {v: i for i, v in enumerate(no_def_allowed)}  # 28 классов

# Префикс: 0 - нет, 1 - "PL", 2 - "PF"
prefix_mapping = {"": 0, "PL": 1, "PF": 2}

# Постфикс: 0 - нет, 1 - "BN", 2 - "RB", 3 - "RD"
postfix_mapping = {"": 0, "BN": 1, "RB": 2, "RD": 3}

def parse_condition(cond_str):
    """
    Парсит строку condition и возвращает словарь с метками для всех задач.
    Возвращает:
      defect: 0 или 1
      preservation_defect: int (0..7) для дефектных монет, если не применимо, 0
      preservation_no_defect: int (0..27) для монет без дефектов, иначе 0
      prefix: int (0..2) – 0: нет, 1: PL, 2: PF; применяется для монет без дефектов и оценки >53, иначе 0
      postfix: int (0..3) – 0: нет, 1: BN, 2: RB, 3: RD; если не применимо, 0
    """
    cond_str = cond_str.strip()
    # Если присутствует подстрока "det" (регистр не важен), считаем монету с дефектами
    if "det" in cond_str.lower():
        # Найдем ключ из defective_mapping, если он встречается в строке
        defect = 1
        pres_def = 0
        for key in defective_mapping:
            if key.lower() in cond_str.lower():
                pres_def = defective_mapping[key]
                break
        # Для монет с дефектами остальные поля не применимы
        pres_no_def = 0
        pref = 0
        post = 0
    else:
        # Монета без дефектов
        defect = 0
        # Предположим, что возможен префикс (PL или PF) если строка начинается с ним; иначе – нет.
        pref = 0
        numeric_part = cond_str[2:]
        # Проверяем наличие префикса (PL или PF) – если первые 2 символа буквы
        if cond_str.startswith("PL"):
            pref = prefix_mapping["PL"]
            numeric_part = cond_str[2:]
        elif cond_str.startswith("PF"):
            pref = prefix_mapping["PF"]
            numeric_part = cond_str[2:]
        else:
            pref = prefix_mapping[""]
        # Теперь проверяем наличие постфикса (BN, RB, RD) в конце строки
        post = 0
        # Если последние 2 символа – буквы и соответствуют одному из вариантов
        if numeric_part.endswith("BN"):
            post = postfix_mapping["BN"]
            numeric_part = numeric_part[:-2]
        elif numeric_part.endswith("RB"):
            post = postfix_mapping["RB"]
            numeric_part = numeric_part[:-2]
        elif numeric_part.endswith("RD"):
            post = postfix_mapping["RD"]
            numeric_part = numeric_part[:-2]
        else:
            post = postfix_mapping[""]
        # Удаляем возможные пробелы
        numeric_part = numeric_part.strip()
        try:
            numeric_val = int(numeric_part)
        except:
            numeric_val = no_def_allowed[0]  # по умолчанию минимальное значение
        # Сохраняем оценку сохранности для монет без дефектов
        pres_no_def = no_def_mapping.get(numeric_val, 0)
        # Для монет без дефектов оценка preservation_defect не используется (ставим 0)
        pres_def = 0
    return {
        "defect": defect,
        "preservation_defect": pres_def,
        "preservation_no_defect": pres_no_def,
        "prefix": pref,
        "postfix": post
    }

In [44]:
print(parse_condition('MS65'))

{'defect': 0, 'preservation_defect': 0, 'preservation_no_defect': 21, 'prefix': 0, 'postfix': 0}


In [60]:
#############################################
# 2. ЗАГРУЗКА ДАННЫХ И ПРЕДОБРАБОТКА DataFrame
#############################################

# Укажите путь к CSV и директории с изображениями
image_dir = r'D:\Python\coin_pages\\'
df = pd.read_csv(r"D:\Python\coin_data_final.csv", index_col='Unnamed: 0')

# Применяем парсинг condition к каждой строке и добавляем новые столбцы
parsed = df['condition'].apply(parse_condition)
df['defect'] = parsed.apply(lambda x: x["defect"])
df['preservation_defect'] = parsed.apply(lambda x: x["preservation_defect"])
df['preservation_no_defect'] = parsed.apply(lambda x: x["preservation_no_defect"])
df['prefix'] = parsed.apply(lambda x: x["prefix"])
df['postfix'] = parsed.apply(lambda x: x["postfix"])

df=df.head(5000)

In [61]:
#############################################
# 3. Генератор данных с patch-based подходом и multi-task label
#############################################

# Параметры изображений и патчей
ORIGINAL_SIZE = (1024, 1024)   # исходное разрешение
PATCH_SIZE = (224, 224)        # размер патча
GRID_SIZE = (1,1)          # число патчей, извлекаемых из каждого изображения

# Маска для круга вписанного в квадрат

def apply_circular_mask(img_array):
    """
    Принимает изображение в виде numpy-массива (H, W, 3).
    Создает маску — круг с центром в середине изображения и радиусом = min(H,W)/2.
    Пиксели вне круга обнуляются.
    """
    H, W, _ = img_array.shape
    # Центр изображения
    center_y, center_x = H / 2, W / 2
    # Радиус круга — половина минимального размера
    radius = min(H, W) / 2
    # Создаем индексы по строкам и столбцам
    Y, X = np.ogrid[:H, :W]
    dist_from_center = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    mask = dist_from_center <= radius  # логическая маска
    # Приводим mask к размерности (H,W,1) и умножаем
    mask = np.expand_dims(mask, axis=-1).astype(img_array.dtype)
    return img_array * mask



class CoinMultiTaskDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size, original_size, patch_size, grid_size, image_dir, shuffle=True):
        """
        dataframe: DataFrame с колонками: 'number', 'avers', 'revers' и метками (например, 'defect', 'preservation_defect', ...).
        original_size: исходное разрешение изображений, например, (1024, 1024)
        patch_size: размер патча, например, (224, 224)
        grid_size: кортеж (rows, cols), определяющий количество патчей по вертикали и горизонтали (например, (4,4))
        image_dir: путь к директории с изображениями
        """
        self.dataframe = dataframe.reset_index(drop=True)
        self.batch_size = batch_size
        self.original_size = original_size
        self.patch_size = patch_size
        self.grid_size = grid_size  # например, (4,4)
        self.image_dir = image_dir
        self.shuffle = shuffle
        self.indices = np.arange(len(self.dataframe))
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))
    
    def _extract_grid_patches(self, img_array):
        """
        Разбивает изображение (уже замаскированное) на равномерную сетку патчей.
        Возвращает патчи в виде numpy массива формы (rows*cols, patch_h, patch_w, 3).
        """
        H, W, _ = img_array.shape
        rows, cols = self.grid_size
        patch_h, patch_w = self.patch_size
        
        # Если grid_size == 1, берем единственный патч из верхнего левого угла
        if rows == 1:
            row_step = 0
        else:
            row_step = (H - patch_h) // (rows - 1)
        if cols == 1:
            col_step = 0
        else:
            col_step = (W - patch_w) // (cols - 1)
            
        patches = []
        for i in range(rows):
            for j in range(cols):
                y = i * row_step
                x = j * col_step
                # Если выход за границы, обрезаем (обычно не случается при корректном расчете)
                patch = img_array[y:y+patch_h, x:x+patch_w, :]
                patches.append(patch)
        return np.array(patches)  # shape: (rows*cols, patch_h, patch_w, 3)
    
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_data = self.dataframe.iloc[batch_indices]
        
        revers_batch, avers_batch = [], []
        # Собираем метки для всех задач
        labels_defect = []
        labels_pres_def = []
        labels_pres_no_def = []
        labels_prefix = []
        labels_postfix = []
        
        for _, row in batch_data.iterrows():
            # Полные пути к изображениям
            revers_path = os.path.join(self.image_dir, row['revers'])
            avers_path  = os.path.join(self.image_dir, row['avers'])
            
            try:
                img_revers = load_img(revers_path, target_size=self.original_size)
                img_avers  = load_img(avers_path, target_size=self.original_size)
            except Exception as e:
                print("Ошибка загрузки:", e)
                continue
            img_revers = img_to_array(img_revers)
            img_avers  = img_to_array(img_avers)
            
            # Применяем круговую маску, чтобы учитывать только область монеты
            img_revers = apply_circular_mask(img_revers)
            img_avers  = apply_circular_mask(img_avers)
            
            # Извлекаем патчи с помощью равномерного разбиения (grid-based extraction)
            revers_patches = self._extract_grid_patches(img_revers)  # форма: (rows*cols, patch_h, patch_w, 3)
            avers_patches  = self._extract_grid_patches(img_avers)
            
            revers_batch.append(revers_patches)
            avers_batch.append(avers_patches)
            
            # Собираем метки (предполагается, что они уже есть в DataFrame)
            labels_defect.append(int(row['defect']))
            labels_pres_def.append(int(row['preservation_defect']))
            labels_pres_no_def.append(int(row['preservation_no_defect']))
            labels_prefix.append(int(row['prefix']))
            labels_postfix.append(int(row['postfix']))
        
        X_revers = np.array(revers_batch)  # shape: (batch_size, rows*cols, patch_h, patch_w, 3)
        X_avers  = np.array(avers_batch)
        
        y = {
            "defect": np.array(labels_defect, dtype=np.float32),
            "preservation_defect": np.array(labels_pres_def, dtype=np.int32),
            "preservation_no_defect": np.array(labels_pres_no_def, dtype=np.int32),
            "prefix": np.array(labels_prefix, dtype=np.int32),
            "postfix": np.array(labels_postfix, dtype=np.int32)
        }
        
        return (X_revers, X_avers), y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

Кастомные метрики.

In [102]:
def ordinal_mae(y_true, y_pred):
    """
    Кастомная метрика для вычисления MAE для ordinal задач.
    y_pred имеет форму (batch, num_classes) – распределение вероятностей.
    y_true имеет форму (batch,) или (batch,1) – скалярные истинные метки.
    Вычисляем argmax по y_pred и затем среднюю абсолютную ошибку.
    """
    # Приводим y_true к форме (batch,) и преобразуем в int32
    y_true = tf.cast(tf.squeeze(y_true), tf.int32)
    # Получаем предсказанные метки как argmax
    y_pred_labels = tf.argmax(y_pred, axis=-1, output_type=tf.int32)
    # Вычисляем абсолютную ошибку
    error = tf.abs(tf.cast(y_true, tf.float32) - tf.cast(y_pred_labels, tf.float32))
    return tf.reduce_mean(error)

def qwk_metric(y_true, y_pred):
    """
    Вычисляет Quadratic Weighted Kappa (QWK) между истинными метками и предсказаниями.
    y_true: форма (batch,) или (batch,1) с целочисленными метками.
    y_pred: распределение вероятностей (batch, num_classes).
    """
    y_true = tf.cast(tf.squeeze(y_true), tf.int32)
    y_pred_labels = tf.argmax(y_pred, axis=-1, output_type=tf.int32)

    def sklearn_qwk(y_true_np, y_pred_np):
        return np.float32(cohen_kappa_score(y_true_np, y_pred_np, weights='quadratic'))
    
    qwk = tf.py_function(sklearn_qwk, [y_true, y_pred_labels], tf.float32)
    return qwk
    
# ---------------------------
# 4. Взвешенная бинарная кроссэнтропия для задачи defect
# ---------------------------
def weighted_binary_crossentropy(pos_weight):
    """
    pos_weight – вес для положительного класса.
    Если положительный класс встречается реже, pos_weight > 1.
    """
    def loss(y_true, y_pred):
        # Стандартная бинарная кроссэнтропия
        bce = K.binary_crossentropy(y_true, y_pred)
        # Вес для каждого примера: если y_true == 1, вес = pos_weight, иначе 1.
        weight_vector = y_true * pos_weight + (1.0 - y_true)
        return K.mean(bce * weight_vector)
    return loss

# ---------------------------
# 2. Функция потерь, основанная на Earth Mover’s Distance (EMD) для ordinal задач
# (здесь мы суммируем разницу между накопительными распределениями)
# ---------------------------
def ordinal_emd_loss(num_classes):
    """
    num_classes – число классов в данной задаче.
    Эта функция предполагает, что y_true имеет форму (batch, 1) (скалярные метки),
    а y_pred – распределение вероятностей по классам (batch, num_classes).
    """
    def loss(y_true, y_pred):
        # Преобразуем y_true в one-hot (batch, num_classes)
        y_true_onehot = tf.one_hot(tf.cast(tf.squeeze(y_true), tf.int32), depth=num_classes)
        # Вычисляем накопительные суммы (CDF)
        cdf_true = tf.math.cumsum(y_true_onehot, axis=1)
        cdf_pred = tf.math.cumsum(y_pred, axis=1)
        # EMD – средняя абсолютная разность между CDF
        return tf.reduce_mean(tf.abs(cdf_true - cdf_pred))
    return loss

In [103]:
#############################################
# 4. МОДЕЛЬ Multi-Task с patch-based подходом
#############################################

def create_patch_feature_extractor(patch_size):
    """
    Создает модель для извлечения признаков из одного патча.
    Использует ResNet50 (без верхней части) + GlobalAveragePooling2D и Dense.
    """
    # Явно приводим размеры к int, на случай если patch_size передается не в виде целых чисел.
    patch_h = int(patch_size[0])
    patch_w = int(patch_size[1])
    
    patch_input = Input(shape=(patch_h, patch_w, 3))
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(patch_h, patch_w, 3))
    base_model.trainable = False
    x = base_model(patch_input)
    x = GlobalAveragePooling2D()(x)
    out = Dense(128, activation='relu')(x)
    return Model(inputs=patch_input, outputs=out, name='patch_feature_extractor')

def create_multi_task_model(patch_size, grid_size):
    """
    Строит модель, которая получает на вход набор патчей для реверса и аверса,
    извлекает общие признаки, агрегирует их и имеет 5 выходов:
      - defect (бинарная)
      - preservation_defect (8 классов)
      - preservation_no_defect (28 классов)
      - prefix (3 класса)
      - postfix (4 класса)
    """
    
    
    # Входы: для каждого изображения набор патчей
    num_patches = grid_size[0] * grid_size[1]   # Это уже int, если grid_size – кортеж чисел
    
    input_revers = Input(shape=(num_patches, patch_size[0], patch_size[1], 3), name='revers_input')
    input_avers  = Input(shape=(num_patches, patch_size[0], patch_size[1], 3), name='avers_input')
    
    # Создаем общий feature extractor для патчей
    patch_extractor = create_patch_feature_extractor(patch_size)
    
    # Применяем его к каждому патчу с помощью TimeDistributed
    revers_features = TimeDistributed(patch_extractor, name='td_revers')(input_revers)  # (batch, patches, 128)
    avers_features  = TimeDistributed(patch_extractor, name='td_avers')(input_avers)    # (batch, patches, 128)
    
    # Агрегируем признаки по патчам (усреднение)
    revers_agg = GlobalAveragePooling1D(name='gap_revers')(revers_features)  # (batch, 128)
    avers_agg  = GlobalAveragePooling1D(name='gap_avers')(avers_features)    # (batch, 128)
    
    # Объединяем признаки двух сторон
    combined = Concatenate(name='combined')([revers_agg, avers_agg])  # (batch, 256)
    shared_fc = Dense(256, activation='relu', name='shared_fc1')(combined)
    shared_fc = Dropout(0.5, name='dropout_shared')(shared_fc)
    
    #############################################
    # Голова 1: Определение наличия дефектов (бинарная классификация)
    defect_out = Dense(1, activation='sigmoid', name='defect')(shared_fc)
    
    #############################################
    # Голова 2: Сохранность для монет с дефектами (8 классов)
    pres_def_fc = Dense(128, activation='relu', name='pres_def_fc')(shared_fc)
    preservation_defect_out = Dense(8, activation='softmax', name='preservation_defect')(pres_def_fc)
    
    #############################################
    # Голова 3: Сохранность для монет без дефектов (28 классов)
    pres_no_def_fc = Dense(128, activation='relu', name='pres_no_def_fc')(shared_fc)
    preservation_no_defect_out = Dense(28, activation='softmax', name='preservation_no_defect')(pres_no_def_fc)
    
    #############################################
    # Голова 4: Префикс (3 класса: нет, PL, PF)
    prefix_fc = Dense(64, activation='relu', name='prefix_fc')(shared_fc)
    prefix_out = Dense(3, activation='softmax', name='prefix')(prefix_fc)
    
    #############################################
    # Голова 5: Постфикс (4 класса: нет, BN, RB, RD)
    postfix_fc = Dense(64, activation='relu', name='postfix_fc')(shared_fc)
    postfix_out = Dense(4, activation='softmax', name='postfix')(postfix_fc)
    
    model = Model(
        inputs=[input_revers, input_avers],
        outputs={
            "defect": defect_out,
            "preservation_defect": preservation_defect_out,
            "preservation_no_defect": preservation_no_defect_out,
            "prefix": prefix_out,
            "postfix": postfix_out
        }
    )



    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss={
            "defect": weighted_binary_crossentropy(pos_weight=5.0),  # задайте pos_weight в зависимости от дисбаланса
            "preservation_defect": ordinal_emd_loss(8),               # 8 классов для монет с дефектами
            "preservation_no_defect": ordinal_emd_loss(28),             # 28 классов для монет без дефектов
            "prefix": "sparse_categorical_crossentropy",              # 3 класса: нет, PL, PF
            "postfix": "sparse_categorical_crossentropy"              # 4 класса: нет, BN, RB, RD
        },
        loss_weights={
            "defect": 1.0,
            "preservation_defect": 1.0,
            "preservation_no_defect": 1.0,
            "prefix": 0.5,    # если эти задачи менее важны, можно снизить их вклад
            "postfix": 0.5
        },
        metrics={
            "defect": tf.keras.metrics.AUC(name="auc"),
            "preservation_defect": [ordinal_mae],
            "preservation_no_defect": [ordinal_mae],
            "prefix": "accuracy",
            "postfix": "accuracy"
        }
    )
    return model

In [104]:
#############################################
# 5. Разбиение данных и создание генераторов
#############################################

# Разбиваем DataFrame на обучающую и валидационную выборки (80/20)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

batch_size = 8

train_gen_instance = CoinMultiTaskDataGenerator(
    dataframe=train_df,
    batch_size=batch_size,
    original_size=ORIGINAL_SIZE,
    patch_size=PATCH_SIZE,
    grid_size=GRID_SIZE,
    image_dir=image_dir,
    shuffle=True
)
val_gen_instance = CoinMultiTaskDataGenerator(
    dataframe=val_df,
    batch_size=batch_size,
    original_size=ORIGINAL_SIZE,
    patch_size=PATCH_SIZE,
    grid_size=GRID_SIZE,
    image_dir=image_dir,
    shuffle=False
)

# Оборачивание генераторов в tf.data.Dataset
def dataset_generator(generator_instance):
    for i in range(len(generator_instance)):
        yield generator_instance[i]

output_signature = (
    (
        tf.TensorSpec(shape=(None, int(GRID_SIZE[0]*GRID_SIZE[1]), PATCH_SIZE[0], PATCH_SIZE[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, int(GRID_SIZE[0]*GRID_SIZE[1]), PATCH_SIZE[0], PATCH_SIZE[1], 3), dtype=tf.float32)
    ),
    {
        "defect": tf.TensorSpec(shape=(None,), dtype=tf.float32),
        "preservation_defect": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "preservation_no_defect": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "prefix": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "postfix": tf.TensorSpec(shape=(None,), dtype=tf.int32)
    }
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_generator(train_gen_instance),
    output_signature=output_signature
).repeat().prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    lambda: dataset_generator(val_gen_instance),
    output_signature=output_signature
).repeat().prefetch(tf.data.AUTOTUNE)

#############################################
# 6. Создание модели и обучение
#############################################

model = create_multi_task_model(PATCH_SIZE, GRID_SIZE)
model.summary()

# Задаем количество шагов на эпоху
steps_per_epoch = len(train_gen_instance)
validation_steps = len(val_gen_instance)

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ revers_input (InputLayer)     │ (None, 1, 224, 224, 3)    │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ avers_input (InputLayer)      │ (None, 1, 224, 224, 3)    │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ td_revers (TimeDistributed)   │ (None, 1, 128)            │      23,849,984 │ revers_input[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ td_avers (TimeDistributed)    │ (None, 1, 128)            │      23,849,984 │ avers_input[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gap_revers                    │ (None, 128)               │               0 │ td_revers[0][0]            │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gap_avers                     │ (None, 128)               │               0 │ td_avers[0][0]             │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ combined (Concatenate)        │ (None, 256)               │               0 │ gap_revers[0][0],          │
│                               │                           │                 │ gap_avers[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared_fc1 (Dense)            │ (None, 256)               │          65,792 │ combined[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_shared (Dropout)      │ (None, 256)               │               0 │ shared_fc1[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ postfix_fc (Dense)            │ (None, 64)                │          16,448 │ dropout_shared[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ prefix_fc (Dense)             │ (None, 64)                │          16,448 │ dropout_shared[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pres_def_fc (Dense)           │ (None, 128)               │          32,896 │ dropout_shared[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pres_no_def_fc (Dense)        │ (None, 128)               │          32,896 │ dropout_shared[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ defect (Dense)                │ (None, 1)                 │             257 │ dropout_shared[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ postfix (Dense)               │ (None, 4)                 │             260 │ postfix_fc[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ prefix (Dense)                │ (None, 3)                 │             19

 Total params: 24,019,820 (91.63 MB)

 Trainable params: 432,108 (1.65 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [105]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    min_delta=0.001,  # Минимальное улучшение, чтобы считать, что метрика улучшилась
    mode='min',       # Режим минимизации (потеря должна уменьшаться)
    restore_best_weights=True
)

# Запускаем обучение
model.fit(
    train_dataset,
    epochs=1,
    steps_per_epoch=steps_per_epoch//50,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

10/10 ━━━━━━━━━━━━━━━━━━━━ 148s 15s/step - defect_auc: 0.5855 - loss: 3.5813 - postfix_accuracy: 0.4434 - prefix_accuracy: 0.1774 - preservation_defect_ordinal_mae: 4.1996 - preservation_no_defect_ordinal_mae: 7.6149 - val_defect_auc: 0.4646 - val_loss: 2.7875 - val_postfix_accuracy: 0.8330 - val_prefix_accuracy: 0.7910 - val_preservation_defect_ordinal_mae: 2.9220 - val_preservation_no_defect_ordinal_mae: 5.6570


In [106]:
model.save('coin_condition_1.keras')

In [108]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Глобальные параметры, используемые при обучении (убедитесь, что они совпадают)
ORIGINAL_SIZE = (1024, 1024)   # исходное разрешение
PATCH_SIZE = (224, 224)        # размер патча
GRID_SIZE = (1, 1)             # равномерная сетка (здесь для примера 1x1; замените на нужное значение, например, (4,4))
NUM_PATCHES = GRID_SIZE[0] * GRID_SIZE[1]  # ожидаемое число патчей

def ordinal_mae(y_true, y_pred):
    """
    Кастомная метрика для вычисления MAE для ordinal задач.
    y_pred имеет форму (batch, num_classes) – распределение вероятностей.
    y_true имеет форму (batch,) или (batch,1) – скалярные истинные метки.
    Вычисляем argmax по y_pred и затем среднюю абсолютную ошибку.
    """
    # Приводим y_true к форме (batch,) и преобразуем в int32
    y_true = tf.cast(tf.squeeze(y_true), tf.int32)
    # Получаем предсказанные метки как argmax
    y_pred_labels = tf.argmax(y_pred, axis=-1, output_type=tf.int32)
    # Вычисляем абсолютную ошибку
    error = tf.abs(tf.cast(y_true, tf.float32) - tf.cast(y_pred_labels, tf.float32))
    return tf.reduce_mean(error)
    
# ---------------------------
# 4. Взвешенная бинарная кроссэнтропия для задачи defect
# ---------------------------
def weighted_binary_crossentropy(pos_weight):
    """
    pos_weight – вес для положительного класса.
    Если положительный класс встречается реже, pos_weight > 1.
    """
    def loss(y_true, y_pred):
        # Стандартная бинарная кроссэнтропия
        bce = K.binary_crossentropy(y_true, y_pred)
        # Вес для каждого примера: если y_true == 1, вес = pos_weight, иначе 1.
        weight_vector = y_true * pos_weight + (1.0 - y_true)
        return K.mean(bce * weight_vector)
    return loss

# ---------------------------
# 2. Функция потерь, основанная на Earth Mover’s Distance (EMD) для ordinal задач
# (здесь мы суммируем разницу между накопительными распределениями)
# ---------------------------
def ordinal_emd_loss(num_classes):
    """
    num_classes – число классов в данной задаче.
    Эта функция предполагает, что y_true имеет форму (batch, 1) (скалярные метки),
    а y_pred – распределение вероятностей по классам (batch, num_classes).
    """
    def loss(y_true, y_pred):
        # Преобразуем y_true в one-hot (batch, num_classes)
        y_true_onehot = tf.one_hot(tf.cast(tf.squeeze(y_true), tf.int32), depth=num_classes)
        # Вычисляем накопительные суммы (CDF)
        cdf_true = tf.math.cumsum(y_true_onehot, axis=1)
        cdf_pred = tf.math.cumsum(y_pred, axis=1)
        # EMD – средняя абсолютная разность между CDF
        return tf.reduce_mean(tf.abs(cdf_true - cdf_pred))
    return loss

model = create_multi_task_model(PATCH_SIZE, GRID_SIZE)
model.load_weights("coin_condition_1.keras")


def apply_circular_mask(img_array):
    """
    Принимает изображение (H, W, 3) и обнуляет все пиксели вне круга, вписанного в изображение.
    """
    H, W, _ = img_array.shape
    center_y, center_x = H / 2, W / 2
    radius = min(H, W) / 2
    Y, X = np.ogrid[:H, :W]
    dist_from_center = np.sqrt((X - center_x) ** 2 + (Y - center_y) ** 2)
    mask = dist_from_center <= radius
    mask = np.expand_dims(mask, axis=-1).astype(img_array.dtype)
    return img_array * mask

def extract_grid_patches(img_array, patch_size, grid_size):
    """
    Разбивает изображение (уже замаскированное) на равномерную сетку патчей.
    Возвращает массив патчей формы (rows*cols, patch_h, patch_w, 3).
    """
    H, W, _ = img_array.shape
    rows, cols = grid_size
    patch_h, patch_w = patch_size
    row_step = (H - patch_h) // (rows - 1) if rows > 1 else 0
    col_step = (W - patch_w) // (cols - 1) if cols > 1 else 0

    patches = []
    for i in range(rows):
        for j in range(cols):
            y = i * row_step
            x = j * col_step
            patch = img_array[y:y+patch_h, x:x+patch_w, :]
            patches.append(patch)
    patches = np.array(patches)
    # Если извлечено меньше патчей, чем ожидалось, заполним недостающие копиями первого патча
    if patches.shape[0] < NUM_PATCHES:
        missing = NUM_PATCHES - patches.shape[0]
        if patches.shape[0] > 0:
            extra = np.tile(patches[0:1], (missing, 1, 1, 1))
        else:
            extra = np.zeros((missing, patch_size[0], patch_size[1], 3), dtype=img_array.dtype)
        patches = np.concatenate([patches, extra], axis=0)
    # Если извлечено больше патчей, обрежем
    if patches.shape[0] > NUM_PATCHES:
        patches = patches[:NUM_PATCHES]
    return patches

def preprocess_image_for_prediction(image_path):
    """
    Загружает изображение по image_path, изменяет его размер до ORIGINAL_SIZE,
    применяет круговую маску, равномерно разбивает изображение на патчи,
    и применяет предобработку для ResNet50.
    Возвращает массив патчей формы (NUM_PATCHES, patch_h, patch_w, 3).
    """
    img = load_img(image_path, target_size=ORIGINAL_SIZE)
    img_array = img_to_array(img)
    img_array = apply_circular_mask(img_array)
    patches = extract_grid_patches(img_array, PATCH_SIZE, GRID_SIZE)
    patches = tf.keras.applications.resnet50.preprocess_input(patches)
    return patches

# Обратные маппинги для интерпретации предсказаний
defective_mapping_inv = {
    0: "PF det.",
    1: "UNC det.",
    2: "AU det.",
    3: "XF det.",
    4: "VF det.",
    5: "F det.",
    6: "VG det.",
    7: "G det."
}
no_def_allowed = [4,6,8,10,12,15,20,25,30,35,40,45,50,53,55,58,60,61,62,63,64,65,66,67,68,69,70]
no_def_mapping_inv = {i: str(val) for i, val in enumerate(no_def_allowed)}
prefix_mapping_inv = {0: "", 1: "PL", 2: "PF"}
postfix_mapping_inv = {0: "", 1: "BN", 2: "RB", 3: "RD"}

# Функция, которая принимает пути к изображениям и возвращает оценку монеты
def predict_coin_condition(avers_path, revers_path):
    """
    Принимает пути к изображению аверса и реверса, выполняет предобработку,
    получает предсказание от модели и возвращает словарь с результатами.
    """
    # Получаем патчи для каждого изображения
    patches_avers = preprocess_image_for_prediction(avers_path)  # (NUM_PATCHES, 224, 224, 3)
    patches_revers = preprocess_image_for_prediction(revers_path)
    
    # Добавляем размерность батча: итоговая форма (1, NUM_PATCHES, 224, 224, 3)
    X_avers = np.expand_dims(patches_avers, axis=0)
    X_revers = np.expand_dims(patches_revers, axis=0)
    
    # Порядок входов: проверьте, какой порядок использовался при обучении.
    # Здесь предполагается, что модель ожидает [revers, avers]
    preds = model.predict([X_revers, X_avers], verbose=0)
    
    # Распаковываем предсказания (адаптируйте под вашу модель)
    pred_defect = preds["defect"][0][0]
    pred_pres_def = preds["preservation_defect"][0]
    pred_pres_no_def = preds["preservation_no_defect"][0]
    pred_prefix = preds["prefix"][0]
    pred_postfix = preds["postfix"][0]
    
    has_defect = pred_defect >= 0.5
    if has_defect:
        cls_idx = np.argmax(pred_pres_def)
        condition_str = defective_mapping_inv.get(cls_idx, "Неопределено")
    else:
        cls_idx = np.argmax(pred_pres_no_def)
        numeric_str = no_def_mapping_inv.get(cls_idx, "")
        if numeric_str and int(numeric_str) > 53:
            pref_idx = np.argmax(pred_prefix)
            prefix_str = prefix_mapping_inv.get(pref_idx, "")
        else:
            prefix_str = ""
        post_idx = np.argmax(pred_postfix)
        postfix_str = postfix_mapping_inv.get(post_idx, "")
        condition_str = f"{prefix_str}{numeric_str}{postfix_str}"
    
    result = {
        "defect_probability": float(pred_defect),
        "condition": condition_str
    }
    return result

In [110]:
predict_coin_condition("D:\\Python\\1.jpg", "D:\\Python\\1.jpg")

{'defect_probability': 0.15566132962703705, 'condition': '64'}